***
### <font color=red>Chapter 4: MedTALN inc. Case Study - Preparing OCI Tenancy and Setup Verification</font>
<p style="margin-left:10%; margin-right:10%;">by <font color=teal> John Doe (typica.ai) </font></p>

***

#### Overview:
This notebook assesses the readiness of our notebook session for the MedTALN inc. case study, performing essential checks to ensure our environment is correctly set up. The checks include:

- **GPU Verification**: Confirms whether or not this notebook session has a GPU attached.
- **Object Storage Authentication**: Validates authentication using the notebook session's Resource Principal, which is the recommended approach.
- **Object Creation in Bucket**: Tests the ability to create a dummy text file object in our Labeling Datasets Bucket, verifying both access and functionality.

##### GPU Verification
Confirms whether or not this notebook session has a GPU attached.

In [1]:
import torch

# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory:')
    print('Total:', round(torch.cuda.get_device_properties(0).total_memory/1024**3,1), 'GB')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Reserved:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')


Using device: cpu


##### Object Storage Authentication
Validates authentication using the notebook session's Resource Principal, which is the recommended approach.

In [2]:
import oci

# Initialize OCI Object Storage Client with notebook session's resource principal
signer = oci.auth.signers.get_resource_principals_signer()
object_storage_client = oci.object_storage.ObjectStorageClient(config={}, signer=signer)
object_storage_client

##### Object Creation in Bucket
Tests the ability to create a dummy text file object in our Labeling Datasets Bucket, verifying both access and functionality.

In [3]:
# Initialize Object Storage bucket infos
namespace = object_storage_client.get_namespace().data #yz2wwgkgt8eh
bucket_name = "labelling-datasets-bkt"

# Initialize Object Storage object's name and body
object_name = "dummy.txt"
object_data = b"Dummy text file for setup check - to be deleted."

#create the object in the bucket
obj = object_storage_client.put_object(
    namespace,
    bucket_name,
    object_name,
    object_data)

# Check the creation of the dummy file in the bucket
list_objects_response = object_storage_client.list_objects(
    namespace_name=namespace,
    bucket_name=bucket_name,
    fields="timeCreated"
)

# Get the data from response
print(f"File named {list_objects_response.data.objects[0].name} created on {list_objects_response.data.objects[0].time_created}")


File named dummy.txt created on 2024-08-15 20:07:17.629000+00:00
